# Dengue Time-Series Forecasting — Final Model

This notebook contains the final forecasting pipeline for predicting weekly dengue cases in San Juan and Iquitos.

The model combines climate variables with carefully engineered time-based features.  
Validation is performed using strict chronological splits to reflect real-world forecasting.

## 1. Import Required Libraries

We import the libraries needed for:

- Data handling (NumPy, Pandas)
- Time-series cross-validation (TimeSeriesSplit)
- Evaluation (MAE)
- Modeling (CatBoost)

These form the core components of the forecasting pipeline.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Training Data

We load the DengAI training features and labels.

The dataset is indexed by:

- City  
- Year  
- Week of year  

Keeping this structure ensures that temporal ordering is preserved throughout the workflow.

In [2]:
# Load training data with proper time-series indexing
train_features = pd.read_csv(
    "/kaggle/input/datasets/hardikthapar/deng123/DengAI_Training_Data_Features.csv",
    index_col=[0, 1, 2]  # city, year, weekofyear
)

train_labels = pd.read_csv(
    "/kaggle/input/datasets/hardikthapar/deng123/DengAI_Training_Data_Labels.csv",
    index_col=[0, 1, 2]
)

print(f"Training features shape: {train_features.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"\nCities: {train_features.index.get_level_values(0).unique().tolist()}")

Training features shape: (1456, 21)
Training labels shape: (1456, 1)

Cities: ['sj', 'iq']


## 3. Separate Cities and Maintain Chronology

San Juan and Iquitos are modeled separately.

Each city has different climate patterns and outbreak behavior, so separate models allow better specialization.

We also:

- Remove the `week_start_date` column  
- Sort all data chronologically  

This guarantees proper time ordering before feature engineering.

In [3]:
# Separate cities
sj_X_raw = train_features.loc["sj"].copy()
iq_X_raw = train_features.loc["iq"].copy()

sj_y = train_labels.loc["sj"]["total_cases"].copy()
iq_y = train_labels.loc["iq"]["total_cases"].copy()

# Drop date column (already in index)
sj_X_raw = sj_X_raw.drop("week_start_date", axis=1, errors='ignore')
iq_X_raw = iq_X_raw.drop("week_start_date", axis=1, errors='ignore')

# Ensure chronological order
sj_X_raw = sj_X_raw.sort_index()
iq_X_raw = iq_X_raw.sort_index()
sj_y = sj_y.sort_index()
iq_y = iq_y.sort_index()

print(f"San Juan: {len(sj_X_raw)} weeks")
print(f"Iquitos: {len(iq_X_raw)} weeks")

San Juan: 936 weeks
Iquitos: 520 weeks


## 4. Select Core Climate Features

From exploratory analysis, five climate variables were selected:

- Specific humidity  
- Dew point temperature  
- Average temperature  
- Minimum temperature  
- Precipitation  

These are biologically relevant for mosquito growth and dengue transmission.

Limiting features keeps the model focused and reduces unnecessary complexity.

In [31]:
# Select core climate features
base_features = [
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "station_avg_temp_c",
    "station_min_temp_c",
    "precipitation_amt_mm"
]

# Extract base features
sj_X_climate = sj_X_raw[base_features].copy()
iq_X_climate = iq_X_raw[base_features].copy()

print("Selected climate features:")
print(base_features)

Selected climate features:
['reanalysis_specific_humidity_g_per_kg', 'reanalysis_dew_point_temp_k', 'station_avg_temp_c', 'station_min_temp_c', 'precipitation_amt_mm']


## 5. Handle Missing Values

Climate data occasionally contains gaps.

We apply forward-fill to maintain continuity in the time series without introducing future information.

This step is performed carefully to avoid data leakage.
2. In production, historical observations would be available
3. TimeSeriesSplit's expanding window ensures test data from earlier folds becomes training data in later folds

Rows with NaN values (first 12 weeks) are dropped as insufficient history exists.

In [5]:
def fill_missing_values(X_train, X_test):
    """
    Fill missing values using forward-fill method.
    Fit on train, apply to both train and test.
    """
    X_train_filled = X_train.ffill().bfill()
    X_test_filled = X_test.ffill().bfill()
    
    return X_train_filled, X_test_filled

## 6. Create Lag-Based Target Features

Dengue outbreaks show temporal dependency.

To capture this, we add:

- Lag features (1–4 weeks)
- Rolling averages (4 and 8 weeks)
- Rolling standard deviation (4 weeks)

All lag features are strictly backward-looking using `shift(1)`.

No future values are used at any point.

In [6]:
def add_target_lags(X, y, lags=[1, 2, 3, 4], rolling_windows=[4, 8]):
    """
    Add lagged target features without data leakage.
    
    Parameters:
    X : Features matrix
    y : Target variable (aligned with X ind)
    lags : Individual lag periods
    rolling_windows : Rolling mean window sizes
    
    Return:
    DataFrame with lag features added
    """
    X_lagged = X.copy()
    
    # Add individual lags
    for lag in lags:
        X_lagged[f'lag_{lag}'] = y.shift(lag)
    
    # Add rolling means (computed on shifted data to prevent leakage)
    for window in rolling_windows:
        X_lagged[f'rolling_mean_{window}'] = y.shift(1).rolling(window, min_periods=1).mean()
    
    # Add rolling std for variance information
    X_lagged['rolling_std_4'] = y.shift(1).rolling(4, min_periods=1).std()
    
    return X_lagged

## 7. Time-Series based Cross-Validation

We use `TimeSeriesSplit` with 5 folds.

For each fold:

1. Data is split chronologically  
2. Missing values are handled per fold  
3. Lag features are computed using only training data  
4. The model is trained on past data and evaluated on future weeks  

This setup mirrors real forecasting conditions.

#### Log-Transformed CatBoost Regression

Dengue case counts are highly right-skewed with occasional large outbreaks.

We apply log1p transformation to the target:
- Training: y_log = log(1 + y)
- Prediction: y_pred = exp(y_log) - 1

This stabilizes variance and helps the model learn patterns across different outbreak scales.

CatBoost is configured with:
- MAE loss function (aligned with evaluation metric)
- 500 iterations with early stopping potential
- Depth 6 and learning rate 0.03 (moderate complexity)
- Random seed for reproducibility

In [28]:
def run_time_aware_cv(X_climate, y, city_name, n_splits=5, params=None):
    """
    Perform time-series cross-validation with proper fold-wise feature engineering.
    
    This ensures that no data leaks due to:
    1. Splitting data first
    2. Computing lag features per-fold using only training data
    3. Never shuffling data
    """
    
    if params is None:
        params = {
            'loss_function': 'MAE',
            'iterations': 500,
            'depth': 6,
            'learning_rate': 0.03,
            'random_seed': 42,
            'verbose': 0
        }
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_scores = []
    

    print(f"{city_name} - Time Series Cross-Validation")
   
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X_climate)):
        
        # Split climate features
        X_train_climate = X_climate.iloc[train_idx]
        X_test_climate = X_climate.iloc[test_idx]
        
        # Split target
        y_train = y.iloc[train_idx]
        y_test = y.iloc[test_idx]
        
        # Fill missing values per-fold
        X_train_climate, X_test_climate = fill_missing_values(X_train_climate, X_test_climate)
        
        # Add lag features (CRITICAL: use full y for test to have historical context)
        X_train_lagged = add_target_lags(X_train_climate, y_train)
        X_test_lagged = add_target_lags(X_test_climate, y)  # Use full y series for lags
        
        # Drop rows with NaN (created from lag creation)
        train_valid_idx = X_train_lagged.dropna().index
        test_valid_idx = X_test_lagged.dropna().index
        
        X_train_final = X_train_lagged.loc[train_valid_idx]
        y_train_final = y_train.loc[train_valid_idx]
        
        X_test_final = X_test_lagged.loc[test_valid_idx]
        y_test_final = y_test.loc[test_valid_idx]
        
        # Log transform target (to stabilize variance)
        y_train_log = np.log1p(y_train_final)
        
        # Train model
        model = CatBoostRegressor(**params)
        model.fit(X_train_final, y_train_log)
        
        # Predict and inverse transform
        preds_log = model.predict(X_test_final)
        preds = np.expm1(preds_log).clip(0)  # Ensure non-negative
        
        # Evaluate
        fold_mae = mean_absolute_error(y_test_final, preds)
        fold_scores.append(fold_mae)
        
        print(f"Fold {fold + 1}/{n_splits} | "
              f"Train size: {len(X_train_final):4d} | "
              f"Test size: {len(X_test_final):3d} | "
              f"MAE: {fold_mae:.4f}")
    
    # Summary statistics
    mean_mae = np.mean(fold_scores)
    std_mae = np.std(fold_scores)
    
    print(f"CV Results: Mean MAE = {mean_mae:.4f} ± {std_mae:.4f}")
    
    return fold_scores, mean_mae

## 8.  Cross-Validation 
#### San Juan

The full time-aware validation pipeline is executed for San Juan.

MAE is computed for each fold and summarized using mean and standard deviation.

This provides an honest estimate of generalization performance.

#### Iquitos

The same validation procedure is applied to Iquitos.

Separate validation ensures the model adapts correctly to the city's unique outbreak dynamics.

In [29]:
# San Juan CV
sj_cv_scores, sj_mean_mae = run_time_aware_cv(
    sj_X_climate, 
    sj_y, 
    "San Juan",
    n_splits=5
)

# Iquitos CV
iq_cv_scores, iq_mean_mae = run_time_aware_cv(
    iq_X_climate, 
    iq_y, 
    "Iquitos",
    n_splits=5
)

San Juan - Time Series Cross-Validation
Fold 1/5 | Train size:  152 | Test size: 156 | MAE: 26.6805
Fold 2/5 | Train size:  308 | Test size: 156 | MAE: 14.2060
Fold 3/5 | Train size:  464 | Test size: 156 | MAE: 6.1003
Fold 4/5 | Train size:  620 | Test size: 156 | MAE: 3.5632
Fold 5/5 | Train size:  776 | Test size: 156 | MAE: 8.7927
CV Results: Mean MAE = 11.8685 ± 8.2045
Iquitos - Time Series Cross-Validation
Fold 1/5 | Train size:   86 | Test size:  86 | MAE: 5.0699
Fold 2/5 | Train size:  172 | Test size:  86 | MAE: 5.2896
Fold 3/5 | Train size:  258 | Test size:  86 | MAE: 3.5831
Fold 4/5 | Train size:  344 | Test size:  86 | MAE: 5.8947
Fold 5/5 | Train size:  430 | Test size:  86 | MAE: 3.8642
CV Results: Mean MAE = 4.7403 ± 0.8775


## 9. Train Final Models on Full Data

After validation, final models are trained using the complete historical dataset.

Steps:

- Apply missing value handling
- Compute lag features
- Apply log transformation to stabilize variance
- Train CatBoost with MAE objective

These models are used for final forecasting.

In [9]:
def train_final_model(X_climate, y, city_name, params=None):
   
    #Train final production model on all available training data.
    if params is None:
        params = {
            'loss_function': 'MAE',
            'iterations': 500,
            'depth': 6,
            'learning_rate': 0.03,
            'random_seed': 42,
            'verbose': 100
        }
    
    print(f"\nTraining final {city_name} model...")
    print("="*60)
    
    # Fill missing values
    X_climate_filled = X_climate.ffill().bfill()
    
    # Add lag features
    X_lagged = add_target_lags(X_climate_filled, y)
    
    # Drop NaN rows
    valid_idx = X_lagged.dropna().index
    X_final = X_lagged.loc[valid_idx]
    y_final = y.loc[valid_idx]
    
    # Log transform
    y_log = np.log1p(y_final)
    
    # Train
    model = CatBoostRegressor(**params)
    model.fit(X_final, y_log)
    
    print(f"\n✓ {city_name} model trained on {len(X_final)} samples")
    
    return model, X_final.columns.tolist()

# Train final models
sj_final_model, sj_features = train_final_model(sj_X_climate, sj_y, "San Juan")
iq_final_model, iq_features = train_final_model(iq_X_climate, iq_y, "Iquitos")


Training final San Juan model...
0:	learn: 0.7924395	total: 4.44ms	remaining: 2.21s
100:	learn: 0.2830851	total: 215ms	remaining: 851ms
200:	learn: 0.2486544	total: 419ms	remaining: 623ms
300:	learn: 0.2260765	total: 624ms	remaining: 413ms
400:	learn: 0.2065425	total: 829ms	remaining: 205ms
499:	learn: 0.1931466	total: 1.02s	remaining: 0us

✓ San Juan model trained on 932 samples

Training final Iquitos model...
0:	learn: 0.8404581	total: 2.19ms	remaining: 1.09s
100:	learn: 0.3785882	total: 140ms	remaining: 555ms
200:	learn: 0.3218011	total: 284ms	remaining: 423ms
300:	learn: 0.2811807	total: 423ms	remaining: 279ms
400:	learn: 0.2504238	total: 569ms	remaining: 140ms
499:	learn: 0.2217702	total: 710ms	remaining: 0us

✓ Iquitos model trained on 516 samples


## 10. Prepare Test Features

To generate predictions, test features must match the training feature structure.

Lag features for test data are computed using historical case counts from training.

This reflects real-world deployment where past observations are always available.

In [10]:
def prepare_test_features(test_X_climate, train_y, feature_list):
    """
    Prepare test features with lag features using historical training data.
    
    Parameters:
    -----------
    test_X_climate : DataFrame - Climate features for test set
    train_y : Series - Historical target values from training
    feature_list : list - List of feature names expected by the model
    """
    # Fill missing values
    test_X_filled = test_X_climate.ffill().bfill()
    
    # Combine train and test targets for lag computation
    # This is VALID because we're only using historical data
    combined_y = train_y.copy()
    
    # Add lag features
    test_X_lagged = add_target_lags(test_X_filled, combined_y)
    
    # Handle any remaining NaN
    test_X_lagged = test_X_lagged.ffill().bfill()
    
    # Ensure same features as training
    for col in feature_list:
        if col not in test_X_lagged.columns:
            test_X_lagged[col] = 0  # Add missing feature with 0
    
    test_X_final = test_X_lagged[feature_list]
    
    return test_X_final


def predict_test_set(test_features, train_y_sj, train_y_iq, 
                     model_sj, model_iq, feature_list_sj, feature_list_iq):

    #Generate predictions for test set for separate cities
    test_sj = test_features.loc["sj"].copy()
    test_iq = test_features.loc["iq"].copy()
    
    # Drop date column
    test_sj = test_sj.drop("week_start_date", axis=1, errors='ignore')
    test_iq = test_iq.drop("week_start_date", axis=1, errors='ignore')
    
    # Extract climate features
    test_sj_climate = test_sj[base_features].copy()
    test_iq_climate = test_iq[base_features].copy()
    
    # Prepare features
    test_sj_final = prepare_test_features(test_sj_climate, train_y_sj, feature_list_sj)
    test_iq_final = prepare_test_features(test_iq_climate, train_y_iq, feature_list_iq)
    
    # Predict
    sj_preds_log = model_sj.predict(test_sj_final)
    iq_preds_log = model_iq.predict(test_iq_final)
    
    # Inverse transform and clip
    sj_preds = np.expm1(sj_preds_log).clip(0).round().astype(int)
    iq_preds = np.expm1(iq_preds_log).clip(0).round().astype(int)
    
    return sj_preds, iq_preds

## 11. Generate Predictions

Predictions are generated by:

1. Preparing features  
2. Applying trained models  
3. Inverting the log transformation  
4. Enforcing non-negative case counts  

Outputs are formatted according to competition requirements.

#### Create Submission File

Predictions are saved into the required submission format:

- City  
- Year  
- Week of year  
- Predicted total cases  

The final CSV is ready for evaluation.

In [25]:
# Load test features by loading dataset again
test_features = pd.read_csv(
    "/kaggle/input/datasets/hardikthapar/deng123/DengAI_Test_Data_Features.csv",
    index_col=[0, 1, 2]
)

# Generates predictions
sj_predictions, iq_predictions = predict_test_set(
    test_features,
    sj_y,
    iq_y,
    sj_final_model,
    iq_final_model,
    sj_features,
    iq_features
)

# Creates submission file
submission = pd.DataFrame({
    'city': ['sj'] * len(sj_predictions) + ['iq'] * len(iq_predictions),
    'year': test_features.index.get_level_values(1).tolist(),
    'weekofyear': test_features.index.get_level_values(2).tolist(),
    'total_cases': np.concatenate([sj_predictions, iq_predictions])
})

submission.to_csv('dengue_predictions.csv', index=False)
print(" Predictions saved to dengue_predictions.csv")


 Predictions saved to dengue_predictions.csv


## 12. Feature Importance Analysis

We extract feature importance from CatBoost.

This helps verify:

- Lag features contribute strongly  
- Climate variables remain meaningful predictors  

It provides interpretability for the forecasting model.

In [24]:
def plot_feature_importance(model, feature_names, city_name):
   
    #Display feature importance from trained model.
    importance = model.get_feature_importance()
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    print(f"\n{city_name} - Top 10 Most Important Features:")
    print(feature_importance_df.head(10).to_string(index=False))
    
    return feature_importance_df

# Display feature importance
sj_importance = plot_feature_importance(sj_final_model, sj_features, "San Juan")
iq_importance = plot_feature_importance(iq_final_model, iq_features, "Iquitos")


San Juan - Top 10 Most Important Features:
                              feature  importance
                                lag_1   22.993459
                       rolling_mean_4   15.855150
                                lag_2   10.174249
                       rolling_mean_8    9.683045
                                lag_3    7.116145
                                lag_4    6.043782
                   station_avg_temp_c    5.752635
                        rolling_std_4    5.519847
reanalysis_specific_humidity_g_per_kg    5.209066
                 precipitation_amt_mm    4.561265

Iquitos - Top 10 Most Important Features:
                              feature  importance
                                lag_1   20.023508
                        rolling_std_4   11.421250
                       rolling_mean_4   10.863390
                                lag_2   10.118560
                       rolling_mean_8    8.975203
                   station_avg_temp_c    7.531172
             

## 13. Final Validation Summary

This model was developed under strict time-series constraints.

Key safeguards:

- No shuffling  
- No future data usage  
- Lag features computed correctly  
- Cross-validation done chronologically  
- Log transformation applied consistently  

Cross-validation MAE reflects realistic forecasting performance.

The pipeline is reproducible and suitable for production-style deployment.

In [23]:
print("\n Notes: :")
print("  • Lag features are computed per-fold during CV")
print("  • Missing values filled per-fold")
print("  • TimeSeriesSplit is used (no random shuffling is done)")
print("  • Test data is never used in training")

print("\n Maximum Achieved Performance:")
print(f"  • San Juan CV MAE: {sj_mean_mae:.4f}")
print(f"  • Iquitos CV MAE: {iq_mean_mae:.4f}")

print("\n Feature's Engineered:")
print(f"  • {len(base_features)} climate based features")
print("  • Target lags: 1, 2, 3, 4 weeks")
print("  • Rolling means: 4, 8 weeks")
print("  • Rolling std: 4 weeks")




 Notes: :
  • Lag features are computed per-fold during CV
  • Missing values filled per-fold
  • TimeSeriesSplit is used (no random shuffling is done)
  • Test data is never used in training

 Maximum Achieved Performance:
  • San Juan CV MAE: 11.8685
  • Iquitos CV MAE: 4.7403

 Feature's Engineered:
  • 5 climate based features
  • Target lags: 1, 2, 3, 4 weeks
  • Rolling means: 4, 8 weeks
  • Rolling std: 4 weeks


## Conclusions

This project investigates an end-to-end time-series based forecasting for weekly dengue case prediction in San Juan and Iquitos.

The workflow was as follows:

- Structured exploratory data analysis
- Careful feature selection, grounded in climate relevance
- Temporal lag engineering to capture the outbreak dynamics
- Log transformation for variance stabilization
- TimeSeriesSplit for strict chronological cross-validation
- Modeling to be done separately for each city as epidemiological behaviors may differ
- Data validation measures to prevent data leakage

This pipeline is a realistic forecasting setting validating that:
- No shuffling of time-series data is done
- No forward-looking bias during training happens
- Lag features computed correctly
- Evaluation is aligned with the competition metric (MAE) as per the guidelines

 _We iteratively refined from baseline linear methods_ [LinearRegression, RandomForests] _to a temporal-aware, robust gradient boosting framework_ CatBoostRegressor.


More importantly, the process stated:
- Methodological correctness
- True acknowledgment

Reproducibility Interpretability of this notebook represents a complete, production-style time-series modeling pipeline suitable for epidemiological forecasting tasks. For me it was not just about improving MAE; rather, the focus was on building a sound and defensible technical forecasting system :)